# Save stitched XRD and XRF maps from flyscans

This notebook stitch and save XRF and XRD maps from a flyscan. It requires data from the `.h5` masterfile, the '_falconx.h5' and the `_pilatus_integrated.h5` file.

The stitched file will then be saved in `/process/stitced_map`

### Load data and plot average XRD for the map

In [ ]:
%matplotlib widget
import os
import sys
import h5py
import numpy as np
import matplotlib.pyplot as plt
#To import DanMAX from the folder above:
sys.path.append('../')
import DanMAX as DM
style = DM.darkMode(style_dic={'figure.figsize':'large'})

In [ ]:
# Define scan location:
#Note that scans_sets must be a list of lists, the first entry in each sublist is a list of scans. then XRD boolean and XRF boolean !
scan_sets = [
    [list(range(XXX,XXX)), True, False],
    [list(range(XXX,XXX)), True, True],
]

proposal,visit=DM.getCurrentProposal()

#Select ranges to load (in the unit the data were integrated with)
#useful for reducing the size of large datasets
xrd_range = None
azi_range = None

for scans in scan_sets:
    
    print(f'Loading scans {scans[0][0]}–{scans[0][-1]}')
    XRD = scans[1]
    XRF = scans[2]

    #Load data
    maps = DM.mapping.stitchScans(scans[0],XRF=XRF,XRD=XRD,proposal=proposal,visit=visit,xrd_range=xrd_range,azi_range=azi_range)
    #Apply I0 correction
    if not maps['xrd_map'] is None:
        maps['xrd_map'] = (maps['xrd_map'].transpose(2,0,1)/ maps['I0_map']).transpose(1,2,0)
    if not maps['xrf_map'] is None:
        maps['xrf_map'] = (maps['xrf_map'].transpose(2,0,1)/ maps['I0_map']).transpose(1,2,0)
    
    print(f'Saving scans {scans[0][0]}–{scans[0][-1]}')
    DM.mapping.save_maps(maps,scans[0],proposal=proposal,visit=visit)
    

